<a href="https://colab.research.google.com/github/hongyan99/ConcurMLWorkshop/blob/main/Preprocessed/080423_DNP2_Cell_Annotation_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is  used to time the running of the notebook
import time
start_time = time.time()

In [2]:
# These packages are pre-installed on Google Colab, but are included here to simplify running this notebook locally
%%capture
!pip install matplotlib
!pip install scikit-learn
!pip install numpy
!pip install scipy
!pip install scanpy
!pip install anndata
!pip install leidenalg

In [3]:
# Install packages for analysis and plotting
from scipy.io import mmread
from sklearn.decomposition import TruncatedSVD
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import scanpy as sc
import anndata
import pandas as pd
import traceback
from scipy.sparse import csr_matrix
matplotlib.rcParams.update({'font.size': 22})
%config InlineBackend.figure_format = 'retina'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# For Hongyan only. If your folder has only one "data_file", set this to empty
# string.
double_data_file="/Data_files"

In [7]:
# Google drive root
gd_root = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics"
data_root = f'{gd_root}/H17_LungMk/Data_files{double_data_file}'

# Data roots
patient_root = f"{data_root}/HRA001149"
lungmk_root = f"{gd_root}/H17_LungMk/LungMk"
out_root = f"/content/drive/MyDrive/LungMk"

# Create the directories if they don't exist
!mkdir -p "{patient_root}"

## Standard File Paths

In [8]:
def get_t2g_file_path():
  return f"{data_root}/t2g.txt"

def get_patient_h5ad_path(patient_id):
  return f"{patient_root}/{patient_id}/filtered_normalized_counts.h5ad"

def get_create_leiden_file_path(patient_id):
  patient_folder = f"{out_root}/HRA001149/{patient_id}"
  os.makedirs(patient_folder, exist_ok=True)
  return f"{patient_folder}/{patient_id}_obs_leidenclusters.h5ad"

def get_h5ad_with_t2g_path(patient_id):
  return f"{out_root}/HRA001149/{patient_id}/t2g_final.h5ad"

## Some checks on the H5AD

In [9]:
# Reading the filtered_normalized_counts.h5ad for a specific patient

def read_patient_count(patient_id):
    file_path = f"{patient_root}/{patient_id}/filtered_normalized_counts.h5ad"
    return anndata.read(file_path)

# Just test with one patient_id

patient_id = "HRR339763"
adata = read_patient_count(patient_id)
adata

AnnData object with n_obs × n_vars = 7206 × 60623
    obs: 'n_counts', 'n_genes', 'percent_mito'
    var: 'mean', 'std'
    uns: 'log1p'

In [10]:
adata.obs

,n_counts,n_genes,percent_mito
AAACCTGCAGACAAGC,2139.0,833,0.0
AAACCTGCAGTCGATT,5176.0,1476,0.0
AAACCTGGTCAGAGGT,3950.0,1714,0.0
AAACCTGGTCCTGCTT,5493.0,1371,0.0
AAACCTGGTGATGTGG,4033.0,1737,0.0
...,...,...,...
TTTGTCATCCAAGCCG,4725.0,1584,0.0
TTTGTCATCCCGGATG,2730.0,1182,0.0
TTTGTCATCGCCTGTT,15945.0,3538,0.0
TTTGTCATCTTGAGGT,7810.0,2422,0.0


In [11]:
adata.var_names

Index(['ENSG00000223972.5', 'ENSG00000227232.5', 'ENSG00000278267.1',
       'ENSG00000243485.5', 'ENSG00000284332.1', 'ENSG00000237613.2',
       'ENSG00000268020.3', 'ENSG00000240361.2', 'ENSG00000186092.6',
       'ENSG00000238009.6',
       ...
       'ENSG00000273532.1', 'ENSG00000276351.1', 'ENSG00000275661.1',
       'ENSG00000277856.1', 'ENSG00000275063.1', 'ENSG00000271254.6',
       'ENSG00000275405.1', 'ENSG00000275987.1', 'ENSG00000277475.1',
       'ENSG00000268674.2'],
      dtype='object', length=60623)

In [12]:
adata.obs.columns.tolist()


['n_counts', 'n_genes', 'percent_mito']

## Check Merged_filtered file

In [2]:
# Import pandas library
import pandas as pd

# Read the CSV file into a DataFrame
merged_filtered = pd.read_csv(f'{data_root}/All Sample ID Data/merged_filtered.csv')

# Display the first few rows of the DataFrame
print("Contents of merged_filtered.csv:")
pd.set_option('display.max_colwidth', None)
# Check the size of the DataFrame
print("Size of the DataFrame (rows, columns):", merged_filtered.shape)

merged_filtered.head()

NameError: ignored

## Play with the value type (using string)

In [ ]:
# Load the additional CSV data into a DataFrame
merged_filtered = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/merged_filtered.csv')

# Convert all DataFrame columns to string
merged_filtered = merged_filtered.astype(str)

# Save the DataFrame
merged_filtered.to_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/merged_filtered_string.csv', index=False)

# Read the CSV file back into a DataFrame from the string format
saved_df = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/merged_filtered_string.csv')

# Inspect the DataFrame
print("Column names:", saved_df.columns.tolist())  # Display the column names
print("DataFrame info:")
saved_df.info()  # Display information about the DataFrame (columns, data types, etc.)
print("First few rows:")
print(saved_df.head())  # Display the first few rows of the DataFrame

Column names: ['folder', 'sample_id', 'strategy', 'tissue', 'age', 'patient_id', 'gender', 'stage']
DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620 entries, 0 to 1619
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   folder      1620 non-null   object 
 1   sample_id   1620 non-null   object 
 2   strategy    1620 non-null   object 
 3   tissue      1620 non-null   object 
 4   age         1617 non-null   float64
 5   patient_id  1620 non-null   object 
 6   gender      1620 non-null   object 
 7   stage       1620 non-null   object 
dtypes: float64(1), object(7)
memory usage: 101.4+ KB
First few rows:
      folder sample_id strategy                        tissue   age  \
0  HRR340265  S-M074-1  RNA-Seq  bronchoalveolar lavage fluid  36.0   
1  HRR340266  S-M074-1  TCR-Seq  bronchoalveolar lavage fluid  36.0   
2  HRR340269    S-M075  RNA-Seq  bronchoalveolar lavage fluid  37.0   
3  HRR340270  

In [ ]:
# Read the CSV file into a DataFrame
merged_filtered = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/merged_filtered.csv')

# Display the first few rows of the DataFrame
print("Contents of merged_filtered.csv:")
merged_filtered.head()

Contents of merged_filtered.csv:


,folder,sample_id,strategy,tissue,age,patient_id,gender,stage
0,HRR340265,S-M074-1,RNA-Seq,bronchoalveolar lavage fluid,36.0,P-M074,male,"mild/moderate, progression"
1,HRR340266,S-M074-1,TCR-Seq,bronchoalveolar lavage fluid,36.0,P-M074,male,"mild/moderate, progression"
2,HRR340269,S-M075,RNA-Seq,bronchoalveolar lavage fluid,37.0,P-M075,female,"mild/moderate, progression"
3,HRR340270,S-M075,TCR-Seq,bronchoalveolar lavage fluid,37.0,P-M075,female,"mild/moderate, progression"
4,HRR340271,S-M076-1,RNA-Seq,bronchoalveolar lavage fluid,35.0,P-M076,male,"mild/moderate, progression"


In [ ]:
print(merged_filtered['strategy'].unique())

# Creating OBS Leiden files for all patients for the merged_filtered data

In [1]:
# Load the merged_filtered data
# merged_filtered = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/merged_filtered_string.csv')

# Generate only the first 5 patient files in our folders under HRA001149
# limit = 5

# List of patient IDs
patient_ids = merged_filtered['folder'].unique()

for folder in patient_ids:
  # if limit <= 0:
  #   break

  if folder.startswith('HRR339'):
    try:
        # Reading the filtered_normalized_counts.h5ad for a specific patient
        file_path = get_patient_h5ad_path(folder)

        # Check if the file exists before trying to open it
        if os.path.isfile(file_path):
            # limit = limit - 1
            output_path = get_create_leiden_file_path(folder)
            if os.path.exists(output_path):
              print(f"Patient {folder} already processed. Annotated data is in {output_path}.")
              continue
            adata = anndata.read(file_path)

            # Add 'folder' to adata.obs
            adata.obs['folder'] = folder

            # Merge additional data (from merged_filtered.csv) with single-cell data
            adata.obs = adata.obs.join(merged_filtered.set_index('folder'), on='folder', how='left')

            # Check if neighbors are already computed
            if 'neighbors' not in adata.uns:
                # Compute PCA
                sc.tl.pca(adata, svd_solver='arpack')

                # Compute a neighborhood graph
                sc.pp.neighbors(adata, use_rep='X_pca')

                # Try embedding cells in a 2D space
                try:
                    sc.tl.umap(adata)
                except TypeError:
                    print(f"Skipping UMAP for patient {folder} due to insufficient data.")

            # Run Leiden algorithm
            sc.tl.leiden(adata)

            # Additional check for 'sample_id'
            print("Unique values in 'sample_id' before saving:", adata.obs['sample_id'].unique())

            adata.write(output_path)

            # Print progress
            print(f"Completed processing for patient {folder}. Annotated data saved to {output_path}.")

        else:
            print(f"File does not exist: {file_path}")

    except FileNotFoundError as fnf:
        print(f"File not found: {file_path}")
        traceback.print_exc()

    except Exception as e:
        print(f"An error occurred with folder: {folder}. Error: {str(e)}")

NameError: ignored

In [ ]:
# Read the h5ad file
h5ad_file_path = get_create_leiden_file_path('HRR339728')
adata = anndata.read_h5ad(h5ad_file_path)

# Display the first few rows of the 'obs' attribute (cell annotations)
print("Contents of 'obs' attribute (cell annotations):")
adata.obs

In [14]:
# Display the first few rows of the 'var' attribute (gene annotations)
adata.var

,mean,std
ENSG00000223972.5,0.0,1.0
ENSG00000227232.5,0.0,1.0
ENSG00000278267.1,0.0,1.0
ENSG00000243485.5,0.0,1.0
ENSG00000284332.1,0.0,1.0
...,...,...
ENSG00000271254.6,0.0,1.0
ENSG00000275405.1,0.0,1.0
ENSG00000275987.1,0.0,1.0
ENSG00000277475.1,0.0,1.0


# Adding t2g data to all patient files

In [ ]:

def fetch_row_as_map(df, column_name, value):
    row = df.loc[df[column_name] == value]
    if len(row) > 0:
        return row.iloc[0].to_dict()
    else:
        return None

# Load the common t2g file
gene_trans = pd.read_csv(get_t2g_file_path(), sep='\t', header=None)

gene_trans['transcript_id'] = gene_trans.iloc[:, 0].str.split('.').str[0]
gene_trans['gene_id'] = gene_trans.iloc[:, 1].str.split('.').str[0]
gene_trans['gene_name'] = gene_trans.iloc[:, 2]
gene_trans = gene_trans.drop(gene_trans.columns[[0, 1, 2]], axis=1)

for patient_id in patient_ids:
    # Define the path where you want to save the file
    t2g_final = get_h5ad_with_t2g_path(patient_id)

    if os.path.exists(t2g_final):
      print(f"Patient {patient_id} already processed. The file is located in {t2g_final}")
      continue

    print(f"Processing patient {patient_id}")

    try:
        # Load the patient data
        h5ad_file_path = get_patient_h5ad_path(patient_id)
        adata = sc.read_h5ad(h5ad_file_path)

        transcript_ids = []
        gene_ids = []
        gene_names = []

        matched = 0
        mismatched = 0
        for var_name_id in adata.var_names:
            var_name = var_name_id.split('.')[0]
            gt = fetch_row_as_map(gene_trans, 'gene_id', var_name)
            print('.', end='')

            if gt is not None:
                matched += 1
                transcript_ids.append(gt['transcript_id'])
                gene_ids.append(gt['gene_id'])
                gene_names.append(gt['gene_name'])
            else:
                mismatched += 1
                transcript_ids.append(None)
                gene_ids.append(None)
                gene_names.append(None)

        print(f'\rMatched={matched} and mismatched={mismatched} out of total {adata.n_vars}', end='', flush=True)

        # Ensure the directory exists; if not, create it
        os.makedirs(os.path.dirname(t2g_final), exist_ok=True)

        # Save the data
        adata.write(t2g_final)

    except FileNotFoundError:
        print(f"File not found for patient {patient_id}. Skipping.")

Processing patient HRR340265
File not found for patient HRR340265. Skipping.
Processing patient HRR340266
File not found for patient HRR340266. Skipping.
Processing patient HRR340269
File not found for patient HRR340269. Skipping.
Processing patient HRR340270
File not found for patient HRR340270. Skipping.
Processing patient HRR340271
File not found for patient HRR340271. Skipping.
Processing patient HRR340272
File not found for patient HRR340272. Skipping.
Processing patient HRR340299
File not found for patient HRR340299. Skipping.
Processing patient HRR340300
File not found for patient HRR340300. Skipping.
Processing patient HRR340303
File not found for patient HRR340303. Skipping.
Processing patient HRR340304
File not found for patient HRR340304. Skipping.
Processing patient HRR340307
File not found for patient HRR340307. Skipping.
Processing patient HRR340308
File not found for patient HRR340308. Skipping.
Processing patient HRR340311
File not found for patient HRR340311. Skipping.

In [ ]:
# Define the file path
file_path = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/t2g_final.h5ad"

# Load the Anndata object from the h5ad file
adata = anndata.read_h5ad(file_path)

# Inspect the adata.obs attribute
print("adata.obs:")
adata.obs

adata.obs:


,n_counts,n_genes,percent_mito,sample_id,cell_type,major_type
AAACCTGCAGACAAGC,2139.0,833,0.0,S-M042-2,T_CD4_c07-AHNAK,CD4
AAACCTGCAGTCGATT,5176.0,1476,0.0,S-M042-2,T_CD4_c05-FOS,CD4
AAACCTGGTCAGAGGT,3950.0,1714,0.0,S-M042-2,T_CD8_c08-IL2RB,CD8
AAACCTGGTCCTGCTT,5493.0,1371,0.0,S-M042-2,T_CD4_c01-LEF1,CD4
AAACCTGGTGATGTGG,4033.0,1737,0.0,S-M042-2,T_CD8_c08-IL2RB,CD8
...,...,...,...,...,...,...
TTTGTCATCCAAGCCG,4725.0,1584,0.0,S-M042-2,Mono_c3-CD14-VCAN,Mono
TTTGTCATCCCGGATG,2730.0,1182,0.0,S-M037,B_c03-CD27-AIM2,B
TTTGTCATCGCCTGTT,15945.0,3538,0.0,S-M042-2,DC_c2-CD1C,DC
TTTGTCATCTTGAGGT,7810.0,2422,0.0,S-M042-2,Mono_c3-CD14-VCAN,Mono


In [ ]:
# Inspect the adata.var attribute
print("\nadata.var:")
adata.var


adata.var:


,mean,std,transcript_id,gene_id,gene_name
ENSG00000223972.5,0.000420,0.021666,ENST00000456328,ENSG00000223972,DDX11L1
ENSG00000227232.5,0.021901,0.156124,ENST00000488147,ENSG00000227232,WASH7P
ENSG00000278267.1,0.000000,1.000000,ENST00000619216,ENSG00000278267,MIR6859-1
ENSG00000243485.5,0.000000,1.000000,ENST00000473358,ENSG00000243485,MIR1302-2HG
ENSG00000284332.1,0.000000,1.000000,ENST00000607096,ENSG00000284332,MIR1302-2
...,...,...,...,...,...
ENSG00000271254.6,0.007822,0.090114,ENST00000614336,ENSG00000271254,AC240274.1
ENSG00000275405.1,0.000000,1.000000,ENST00000619109,ENSG00000275405,U1
ENSG00000275987.1,0.000000,1.000000,ENST00000618083,ENSG00000275987,U1
ENSG00000277475.1,0.000000,1.000000,ENST00000612315,ENSG00000277475,AC213203.2


# Concatenate patient files

In [ ]:
%%time
# Google drive root
gd_root = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics"

# Data roots
patient_root = f"{gd_root}/H17_LungMk/Data_files/HRA001149"

# Directory to save the combined and intermediate data
data_root = f'{gd_root}/H17_LungMk/Data_files/All Sample ID Data'

folders = os.listdir(patient_root) # get all the folders in the directory

save_interval = 10  # Save every 10 patients, you can adjust this based on your need
counter = 0

for folder in folders:
    file_path = os.path.join(patient_root, folder, f"{folder}_obs_leidenclusters.h5ad")
    if os.path.isfile(file_path): # check if the file exists
        adata = sc.read(file_path)
        if 'adata_all' in locals():  # check if adata_all exists
            adata_all = anndata.concat([adata_all, adata], index_unique=None)  # concatenate the current file to the overall data
        else:
            adata_all = adata

        counter += 1

        # Save intermediate result
        if counter % save_interval == 0:
            adata_all.write(f"{data_root}/obs_leidenclusters_combined_intermediate.h5ad")
            print(f"Saved intermediate data after processing {counter} patients.")

# Save the final combined data
adata_all.write(f"{data_root}/obs_leidenclusters_combined.h5ad")

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Saved intermediate data after processing 10 patients.


/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  

In [ ]:
# Google drive root
gd_root = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics"

# Data roots
patient_root = f"{gd_root}/H17_LungMk/Data_files/HRA001149"

folders = os.listdir(patient_root) # get all the folders in the directory

# Count the total number of patient files
total_files = sum([os.path.isfile(os.path.join(patient_root, folder, f"{folder}_obs_leidenclusters.h5ad")) for folder in folders])

print(f'Total number of patient files: {total_files}')

save_interval = 10  # Save every 10 patients, you can adjust this based on your need
total_runs = total_files // save_interval + (total_files % save_interval != 0)

print(f'You will need to run the code {total_runs} times.')


Total number of patient files: 39
You will need to run the code 4 times.


In [ ]:
# Define the file path
file_path = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/obs_leidenclusters_combined_intermediate.h5ad"

# Load the Anndata object from the h5ad file
adata = anndata.read_h5ad(file_path)

# Inspect the adata.obs attribute
print("adata.obs:")
adata.obs

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


adata.obs:


,n_counts,n_genes,percent_mito,folder,sample_id,strategy,tissue,age,patient_id,gender,stage,leiden
ACTGATGCACCCTATC,2355.0,1216,0.0,HRR339728,S-M056,RNA-Seq,PBMC,16.0,P-M056,male,"mild/moderate, progression",0
AGGCCACTCTGTCCGT,2903.0,1388,0.0,HRR339728,S-M056,RNA-Seq,PBMC,16.0,P-M056,male,"mild/moderate, progression",1
AGTAGTCCATCCAACA,5963.0,2030,0.0,HRR339728,S-M056,RNA-Seq,PBMC,16.0,P-M056,male,"mild/moderate, progression",1
ATAGACCGTCTTGTCC,2178.0,841,0.0,HRR339728,S-M056,RNA-Seq,PBMC,16.0,P-M056,male,"mild/moderate, progression",0
ATTATCCAGATGTTAG,2556.0,1198,0.0,HRR339728,S-M056,RNA-Seq,PBMC,16.0,P-M056,male,"mild/moderate, progression",0
...,...,...,...,...,...,...,...,...,...,...,...,...
TCGTAGAGTTATGTGC,18525.0,4408,0.0,HRR339737,S-S071-2,RNA-Seq,PBMC,67.0,P-S071,male,"severe/critical, progression",0
TGAGCATTCATCTGTT,7190.0,2374,0.0,HRR339737,S-S071-2,RNA-Seq,PBMC,67.0,P-S071,male,"severe/critical, progression",1
TGAGGGAAGGCGATAC,2754.0,1110,0.0,HRR339737,S-S071-2,RNA-Seq,PBMC,67.0,P-S071,male,"severe/critical, progression",0
TGCGGGTCACGTCTCT,4348.0,2057,0.0,HRR339737,S-S071-2,RNA-Seq,PBMC,67.0,P-S071,male,"severe/critical, progression",1


In [ ]:
# Inspect the adata.var attribute
print("adata.var:")
adata.var

## read gene type lookup (Gene to Transcript- t2g.txt)

We load the gene to transcript so that we can map gene transcript to gene types when needed. This was provided from the Patcher Lab and refers to the GRCh37 human genome

In [ ]:
import pandas as pd

# Read the text file into a DataFrame
t2g_df = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/t2g.txt', sep='\t', header=None)

# Extract the relevant portions of the transcript_id and gene_id
t2g_df['transcript_id'] = t2g_df.iloc[:, 0].str.split('.').str[0]
t2g_df['gene_id'] = t2g_df.iloc[:, 1].str.split('.').str[0]
t2g_df['gene_name'] = t2g_df.iloc[:, 2]

# Drop the original columns
t2g_df = t2g_df.drop(t2g_df.columns[[0, 1, 2]], axis=1)

# Overwrite cleaned data back to cleaned_t2g.txt without index
t2g_df.to_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/cleaned_t2g.txt', sep='\t', index=False)

In [ ]:
# Read the file back into a new DataFrame
saved_df = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/cleaned_t2g.txt', sep='\t')

# Inspect the new DataFrame
saved_df.info()
saved_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227368 entries, 0 to 227367
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     227368 non-null  int64 
 1   transcript_id  227368 non-null  object
 2   gene_id        227368 non-null  object
 3   gene_name      227368 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.9+ MB


,Unnamed: 0,transcript_id,gene_id,gene_name
0,0,ENST00000456328,ENSG00000223972,DDX11L1
1,1,ENST00000450305,ENSG00000223972,DDX11L1
2,2,ENST00000488147,ENSG00000227232,WASH7P
3,3,ENST00000619216,ENSG00000278267,MIR6859-1
4,4,ENST00000473358,ENSG00000243485,MIR1302-2HG


In [ ]:
print("DataFrame size:", t2g_df.shape)

DataFrame size: (227368, 3)


# Merge t2g_final and merged_filtered



In the original docs we never actually combined all of the data. We only ever combined this data with the adata.obs from the t2g_final.h5ad file because you can combine all three files at the same time due to the file size incompatibility and if you save as a CSV you can only save either the obs or the var data. So we only ever saved the obs data.

This is too ram intensive.

In [ ]:
# Path to the file
merged_csv = f"{data_root}/All Sample ID Data/merged_filtered.csv"

# Read the CSV file into a DataFrame
merged_filtered = pd.read_csv(merged_csv)

# Path to the file
t2g_file = f"{data_root}/All Sample ID Data/t2g_final.h5ad"

adata = anndata.read_h5ad(t2g_file)

# Function to handle processing in chunks
def process_in_chunks(chunk_size):
    # List to hold chunks
    chunks = []

    # Loop through range that goes from 0 to len of adata in steps of chunk_size
    for i in range(0, len(adata), chunk_size):
        # Convert a chunk of the adata AnnData object to a DataFrame
        chunk = pd.DataFrame(data=adata.X[i:i+chunk_size],
                             columns=adata.var.index,
                             index=adata.obs.index[i:i+chunk_size])

        # Add sample_id to chunk
        chunk['sample_id'] = adata.obs['sample_id'][i:i+chunk_size]

        # Merge chunk with merged_filtered
        merged_chunk = pd.merge(chunk, merged_filtered, on='sample_id')

        # Add merged_chunk to chunks
        chunks.append(merged_chunk)

    # Concatenate all chunks
    merged_df = pd.concat(chunks, ignore_index=True)

    return merged_df

# Call the function with a chunk size that fits in your RAM
merged_df = process_in_chunks(1000)

Now, we are droping columns to see if we can merge the rest

In [ ]:
# Path to the file
merged_csv = f"{data_root}/All Sample ID Data/merged_filtered.csv"

# Read the CSV file into a DataFrame
merged_filtered = pd.read_csv(merged_csv)

# Path to the file
t2g_file = f"{data_root}/All Sample ID Data/t2g_final.h5ad"

adata = anndata.read_h5ad(t2g_file)

# Define the columns of interest
columns_of_interest = ['folder', 'sample_id', 'stage']

# Select only the columns of interest from the merged_filtered DataFrame
merged_filtered = merged_filtered[columns_of_interest]

# Function to handle processing in chunks
def process_in_chunks(chunk_size):
    # List to hold chunks
    chunks = []

    # Loop through range that goes from 0 to len of adata in steps of chunk_size
    for i in range(0, len(adata), chunk_size):
        # Convert a chunk of the adata AnnData object to a DataFrame
        chunk = pd.DataFrame(data=adata.X[i:i+chunk_size],
                             columns=adata.var.index,
                             index=adata.obs.index[i:i+chunk_size])

        # Add sample_id to chunk
        chunk['sample_id'] = adata.obs['sample_id'][i:i+chunk_size]

        # Merge chunk with merged_filtered
        merged_chunk = pd.merge(chunk, merged_filtered, on='sample_id')

        # Add merged_chunk to chunks
        chunks.append(merged_chunk)

    # Concatenate all chunks
    merged_df = pd.concat(chunks, ignore_index=True)

    return merged_df

# Call the function with a chunk size that fits in your RAM
merged_df = process_in_chunks(1000)

still too ram intensive

# read cell type lookup (Cell Annotations.csv.gz)

We load the cell annotation so that we can mapp cell bar codes to cell types when needed. This file was provided directly from the publication.

In [ ]:
# Load cell annotation data
cell_annotation = pd.read_csv(f'{data_root}/GSE158055_cell_annotation.csv.gz', compression='gzip')

In [ ]:
print("Column names:", cell_annotation.columns.tolist())

Column names: ['cellName', 'sampleID', 'celltype', 'majorType']


In [ ]:
print("DataFrame info:")
cell_annotation.info()

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462702 entries, 0 to 1462701
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   cellName   1462702 non-null  object
 1   sampleID   1462702 non-null  object
 2   celltype   1462702 non-null  object
 3   majorType  1462702 non-null  object
dtypes: object(4)
memory usage: 44.6+ MB


In [ ]:
print("First few rows:")
cell_annotation.head()

First few rows:


,cellName,sampleID,celltype,majorType
0,AACAGGGGTCGGATTT-0,S-S070-1,Mono_c1-CD14-CCL3,Mono
1,AACCAACGTCCGAAAG-0,S-S070-1,B_c02-MS4A1-CD27,B
2,AACCTTTGTAGCACGA-0,S-S070-1,B_c01-TCL1A,B
3,AAGCATCTCTATCGCC-0,S-S070-1,Mono_c2-CD14-HLA-DPB1,Mono
4,AATCACGGTCATAAAG-0,S-S070-1,B_c01-TCL1A,B


### Convert to String Data

In the provided code, the DataFrame named cell_annotation is being converted to a CSV file in gzip-compressed format and saved at a specific location on Google Drive. The reason for converting the DataFrame to a CSV file in string format is to facilitate its future merging with another file in the h5ad format. Converting the DataFrame to string format ensures that the data is easily accessible and can be efficiently merged with the h5ad file, which might be a specific requirement or format needed for downstream data analysis or processing. By converting and saving the DataFrame in this manner, the data can be seamlessly integrated into the h5ad file, allowing for more streamlined and convenient data manipulation and analysis in the future.

In [ ]:
# Load cell annotation data
cell_annotation = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/GSE158055_cell_annotation.csv.gz', compression='gzip')

# Convert all DataFrame columns to string
cell_annotation = cell_annotation.astype(str)

# Save the DataFrame
cell_annotation.to_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/GSE158055_cell_annotation_string.csv.gz', compression='gzip', index=False)

In [ ]:
# Read the CSV file back into a DataFrame
file_path = '/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/GSE158055_cell_annotation_string.csv.gz'
saved_df = pd.read_csv(file_path, compression='gzip')

# Inspect the DataFrame
print(saved_df.head())  # Display the first few rows of the DataFrame
print(saved_df.info())  # Display information about the DataFrame (columns, data types, etc.)

             cellName  sampleID               celltype majorType
0  AACAGGGGTCGGATTT-0  S-S070-1      Mono_c1-CD14-CCL3      Mono
1  AACCAACGTCCGAAAG-0  S-S070-1       B_c02-MS4A1-CD27         B
2  AACCTTTGTAGCACGA-0  S-S070-1            B_c01-TCL1A         B
3  AAGCATCTCTATCGCC-0  S-S070-1  Mono_c2-CD14-HLA-DPB1      Mono
4  AATCACGGTCATAAAG-0  S-S070-1            B_c01-TCL1A         B
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462702 entries, 0 to 1462701
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   cellName   1462702 non-null  object
 1   sampleID   1462702 non-null  object
 2   celltype   1462702 non-null  object
 3   majorType  1462702 non-null  object
dtypes: object(4)
memory usage: 44.6+ MB
None


In [ ]:
# Specify the patient's folder
folder = "HRR339763"

# Reading the filtered_normalized_counts.h5ad for a specific patient
file_path = f"/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/{folder}/filtered_normalized_counts.h5ad"

# Check if the file exists before trying to open it
if os.path.isfile(file_path):
    adata = anndata.read(file_path)

    # Check the first 5 entries in cell identifiers of adata.obs
    print("First 5 cell identifiers in adata.obs:")
    print(cell_annotation.iloc[:, 1].head(5))

    # Check the first 5 entries in 'cellName' column of cell_annotation DataFrame
    print("\nFirst 5 cell identifiers in 'cellName' column of cell_annotation DataFrame:")
    print(cell_annotation['cellName'][:5])

    # Check if any cell identifier in adata.obs is present in 'cellName' column of cell_annotation DataFrame
    common_cells = set(adata.obs.index) & set(cell_annotation['cellName'])
    print(f"\nNumber of common cells between adata.obs and 'cellName' column of cell_annotation DataFrame: {len(common_cells)}")
else:
    print(f"File does not exist: {file_path}")

First 5 cell identifiers in adata.obs:
0    S-S070-1
1    S-S070-1
2    S-S070-1
3    S-S070-1
4    S-S070-1
Name: sampleID, dtype: object

First 5 cell identifiers in 'cellName' column of cell_annotation DataFrame:
0    AACAGGGGTCGGATTT
1    AACCAACGTCCGAAAG
2    AACCTTTGTAGCACGA
3    AAGCATCTCTATCGCC
4    AATCACGGTCATAAAG
Name: cellName, dtype: object

Number of common cells between adata.obs and 'cellName' column of cell_annotation DataFrame: 0


# Checking the OBS Leiden Clusters File

This is coming from patient file HRR339763, which is mild/moderate, progression	PBMC	F.

In [ ]:
# Define the path where you want to save the HDF5 file
path_h5ad = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/obs_leidenclusters.h5ad"

# Load the AnnData object from an HDF5 file
adata = sc.read(path_h5ad)

# Display the head of the observations
adata.obs.head()

In [ ]:
adata.var.head()

In [ ]:
# Load the merged_filtered data
merged_filtered = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/merged_filtered_string.csv')

# Load cell annotation data
cell_annotation = pd.read_csv('/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/GSE158055_cell_annotation_string.csv.gz', compression='gzip')

# Remove '-0' at the end of each cellName
cell_annotation['cellName'] = cell_annotation['cellName'].str.rstrip('-0')

# List of patient IDs
patient_ids = merged_filtered['folder'].unique()

# Load gene transcription data
t2g_path = '/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/cleaned_t2g.txt'
t2g_df = pd.read_csv(t2g_path, sep='\t')

# Function to remove version numbers from ensembl IDs
def strip_versions(ensembl_id):
    return ensembl_id.split('.')[0]

for folder in patient_ids:
    try:
        # Reading the filtered_normalized_counts.h5ad for a specific patient
        file_path = f"/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/{folder}/filtered_normalized_counts.h5ad"

        # Check if the file exists before trying to open it
        if os.path.isfile(file_path):
            adata = anndata.read(file_path)

            # Strip version from adata.var gene identifiers
            adata.var.index = adata.var.index.map(strip_versions)

            # Add 'folder' to adata.obs
            adata.obs['folder'] = folder

            # Convert the data type of the second column of adata.obs to string
            adata.obs.iloc[:, 1] = adata.obs.iloc[:, 1].astype(str)

            # Merge cell annotation with single-cell data (adata.obs)
            # adata.obs = adata.obs.reset_index().merge(cell_annotation, how='inner', left_on=adata.obs.iloc[:, 1], right_on='cellName').set_index('index')

            # Merge adata.var with t2g_df
            # adata.var = adata.var.merge(t2g_df, how='inner', left_on=adata.var.index, right_on='gene_id')

            # Merge additional data (from merged_filtered.csv) with single-cell data
            adata.obs = adata.obs.join(merged_filtered.set_index('folder'), on='folder', how='left')

            # Fill missing values in 'sampleID' and 'celltype'
            # adata.obs['sampleID'] = adata.obs['sampleID'].fillna('missing')
            # adata.obs['majorType'] = adata.obs['majorType'].fillna('missing')
            # adata.obs['celltype'] = adata.obs['celltype'].fillna('missing')

            # Check if neighbors are already computed
            if 'neighbors' not in adata.uns:
                # Compute PCA
                sc.tl.pca(adata, svd_solver='arpack')

                # Compute a neighborhood graph
                sc.pp.neighbors(adata, use_rep='X_pca')

                # Try embedding cells in a 2D space
                try:
                    sc.tl.umap(adata)
                except TypeError:
                    print(f"Skipping UMAP for patient {folder} due to insufficient data.")

            # Run Leiden algorithm
            sc.tl.leiden(adata)

            # Additional check for 'sample_id'
            print("Unique values in 'sample_id' before saving:", adata.obs['sample_id'].unique())

            # Save the annotated data
            output_path = f"/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/{folder}/{folder}_obs_leidenclusters.h5ad"
            adata.write(output_path)

            # Print progress
            print(f"Completed processing for patient {folder}. Annotated data saved to {output_path}.")

        else:
            print(f"File does not exist: {file_path}")

    except FileNotFoundError:
        print(f"File not found: {file_path}")

    except Exception as e:
        print(f"An error occurred with folder: {folder}. Error: {str(e)}")


File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340265/filtered_normalized_counts.h5ad
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340266/filtered_normalized_counts.h5ad
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340269/filtered_normalized_counts.h5ad
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340270/filtered_normalized_counts.h5ad
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340271/filtered_normalized_counts.h5ad
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340272/filtered_normalized_counts.h5ad
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR340299/filtered_n

/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


Completed processing for patient HRR339738. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339738/HRR339738_obs_leidenclusters.h5ad.
Unique values in 'sample_id' before saving: ['S-S072-2']
Completed processing for patient HRR339739. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339739/HRR339739_obs_leidenclusters.h5ad.
Unique values in 'sample_id' before saving: ['S-S072-3']
Completed processing for patient HRR339740. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339740/HRR339740_obs_leidenclusters.h5ad.
Unique values in 'sample_id' before saving: ['S-HC008']
Completed processing for patient HRR339741. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339741/HRR339741_obs_leidenclusters.h5ad.
Unique values in 'sample_id' before saving: ['

/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


Completed processing for patient HRR339755. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339755/HRR339755_obs_leidenclusters.h5ad.
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339756/filtered_normalized_counts.h5ad
Unique values in 'sample_id' before saving: ['S-M041-2']
Completed processing for patient HRR339757. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339757/HRR339757_obs_leidenclusters.h5ad.
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339758/filtered_normalized_counts.h5ad
Unique values in 'sample_id' before saving: ['S-M041-2']
Completed processing for patient HRR339759. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339759/HRR339759_obs_leidenclusters.h5ad.
Unique

/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


Completed processing for patient HRR339761. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339761/HRR339761_obs_leidenclusters.h5ad.
File does not exist: /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339762/filtered_normalized_counts.h5ad
Unique values in 'sample_id' before saving: ['S-M042-2']
Completed processing for patient HRR339763. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339763/HRR339763_obs_leidenclusters.h5ad.
Unique values in 'sample_id' before saving: ['S-M042-2']
Completed processing for patient HRR339764. Annotated data saved to /content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339764/HRR339764_obs_leidenclusters.h5ad.
Unique values in 'sample_id' before saving: ['S-M042-2']
Completed processing for patient HRR339765. Annotated data saved to /content/drive/MyDrive/Pate

In [ ]:
# Check if any cell identifier in the second column of adata.obs is present in 'cellName' column of cell_annotation DataFrame
common_cells = set(adata.obs.iloc[:, 1].astype(str)) & set(cell_annotation['cellName'])
print(f"\nNumber of common cells between second column of adata.obs and 'cellName' column of cell_annotation DataFrame: {len(common_cells)}")


Number of common cells between second column of adata.obs and 'cellName' column of cell_annotation DataFrame: 0


In [ ]:
adata

AnnData object with n_obs × n_vars = 124 × 60623
    obs: 'n_counts', 'n_genes', 'percent_mito', 'folder'
    var: 'mean', 'std'
    uns: 'log1p'

In [ ]:
print("Do 'cellName' values match with adata.obs index: ", cell_annotation['cellName'].isin(adata.obs.index).any())
print("Do 'folder' values match with adata.obs index: ", merged_filtered['folder'].isin(adata.obs.index).any())

Do 'cellName' values match with adata.obs index:  False
Do 'folder' values match with adata.obs index:  False


In [ ]:
print(adata.obs.index[:10])

Index(['ACTGTCCCACCATCCT', 'AGCGTCGTCGGCGGTT', 'CCCTCCTCACCGAAAG',
       'CTTTGCGTCCACGAAT', 'TGGTTCCTCCTAGGGC'],
      dtype='object')


In [ ]:
print(cell_annotation['cellName'][:10])
print(merged_filtered['folder'][:10])

0    AACAGGGGTCGGATTT
1    AACCAACGTCCGAAAG
2    AACCTTTGTAGCACGA
3    AAGCATCTCTATCGCC
4    AATCACGGTCATAAAG
5    AATCACGGTGGTTCTA
6    AATGAAGCAACTGGTT
7    AATGAAGCACCAGCGT
8    AATGCCATCACCTCGT
9    AATGGCTCAAGGCTTT
Name: cellName, dtype: object
0    HRR340265
1    HRR340266
2    HRR340269
3    HRR340270
4    HRR340271
5    HRR340272
6    HRR340299
7    HRR340300
8    HRR340303
9    HRR340304
Name: folder, dtype: object


# Checking a new patient file

In [ ]:
# Read the h5ad file
h5ad_file_path = '/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/HRR339748/HRR339748_obs_leidenclusters.h5ad'
adata = anndata.read_h5ad(h5ad_file_path)

# Display the first few rows of the 'obs' attribute (cell annotations)
print("Contents of 'obs' attribute (cell annotations):")
adata.obs.head()


Contents of 'obs' attribute (cell annotations):


,n_counts,n_genes,percent_mito,folder,sample_id,strategy,tissue,age,patient_id,gender,stage,leiden
AAACCTGAGAGGTACC,4908.0,1477,0.0,HRR339748,S-HC011,RNA-Seq,PBMC,35.0,P-HC011,male,"control, control",1
AAACCTGAGCTAGTCT,23539.0,4829,0.0,HRR339748,S-HC011,RNA-Seq,PBMC,35.0,P-HC011,male,"control, control",15
AAACCTGAGCTGCGAA,5384.0,1669,0.0,HRR339748,S-HC011,RNA-Seq,PBMC,35.0,P-HC011,male,"control, control",2
AAACCTGAGTTAAGTG,4231.0,1423,0.0,HRR339748,S-HC011,RNA-Seq,PBMC,35.0,P-HC011,male,"control, control",2
AAACCTGCAATGAAAC,6083.0,1932,0.0,HRR339748,S-HC011,RNA-Seq,PBMC,35.0,P-HC011,male,"control, control",2


In [ ]:
# Display unique stages from the dataset
unique_stages = adata.obs['stage'].unique()
print("Unique stages in the dataset:", unique_stages)

Unique stages in the dataset: ['control, control']
Categories (1, object): ['control, control']


In [ ]:
# Display the first few rows of the 'var' attribute (gene annotations)
adata.var.head()

,mean,std
ENSG00000223972,0.006409,0.091681
ENSG00000227232,0.011358,0.114008
ENSG00000278267,0.000000,1.000000
ENSG00000243485,0.000000,1.000000
ENSG00000284332,0.000000,1.000000


Because the cell annotation file is not conn

In [ ]:
import matplotlib.pyplot as plt

# Set global font size
plt.rcParams.update({'font.size': 14})

# Increase the figure size
fig, ax = plt.subplots(figsize=(10, 10))

# Increase DPI for higher resolution
fig.dpi = 200

# Create a UMAP for major_type with adjusted parameters
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='leiden', frameon=False, ax=ax, show=False)

# Adjust the title size
ax.set_title('leiden', fontsize=20)

# Show the plot
plt.show()

# Merge all patients

In [ ]:
import anndata

# Prepare a list to store individual AnnData objects
adata_list = []

for patient_id in patient_ids:
    try:
        # Path for the annotated data file for each patient
        file_path = f"/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/HRA001149/{patient_id}/{patient_id}_obs_leidenclusters.h5ad"

        # Check if the file exists before trying to open it
        if os.path.isfile(file_path):
            # Load the annotated data for the patient
            adata = anndata.read(file_path)

            # Append the loaded data to the list
            adata_list.append(adata)

            # Print progress
            print(f"Loaded data for patient {patient_id} from {file_path}.")

        else:
            print(f"File does not exist: {file_path}")

    except FileNotFoundError:
        print(f"File not found: {file_path}")

# Concatenate all individual AnnData objects into one
adata_all = anndata.concat(adata_list)

# Save the concatenated data
output_path = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All_Patients_obs_leidenclusters.h5ad"
adata_all.write(output_path)

print(f"All patients data saved to {output_path}.")


## Cell Quality
Check the distribution of the number of counts and genes per cell. This can be done with histograms or violin plots. Cells with an extremely low or high number of counts or genes can be indicative of poor-quality cells or doublets, respectively.

Cell Quality: Check the distribution of the number of counts and genes per cell. This can be done with histograms or violin plots. Cells with an extremely low or high number of counts or genes can be indicative of poor-quality cells or doublets, respectively.

Gene Detection: Examine the number of cells in which each gene is detected. Genes that are detected in a very low number of cells may be unreliable and could be considered for filtering.

Mitochondrial Content: The proportion of mitochondrial counts to total counts per cell can be a good indicator of cell stress. Cells with high mitochondrial content can be indicative of apoptotic or broken cells.

Doublet Detection: Doublets, or multiple cells captured in the same droplet, can complicate the analysis. There are several computational tools available for doublet detection such as Scrublet.

Batch effects: If your data was generated across multiple batches, it's critical to check for batch effects. Visualization techniques like t-SNE or UMAP can be useful here.

Marker Genes: For identified cell clusters, check the expression of known marker genes. This can validate that the clustering is biologically meaningful.

Differential Expression: Perform differential expression analysis between clusters to identify defining marker genes.

Data Normalization: Check if the data normalization was appropriate. Different datasets require different normalization techniques.

In [ ]:
# Calculate the total counts per cell
adata.obs['n_counts'] = adata.X.sum(axis=1)

# Calculate the total number of genes per cell
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1)

In [ ]:
# Histogram of the total counts per cell
sns.histplot(data=adata.obs, x='n_counts', bins=50)
plt.title('Total counts per cell')
plt.show()

# Histogram of the total number of genes per cell
sns.histplot(data=adata.obs, x='n_genes', bins=50)
plt.title('Total number of genes per cell')
plt.show()

# Violin plot of the total counts per cell
sns.violinplot(data=adata.obs, y='n_counts')
plt.title('Total counts per cell')
plt.show()

# Violin plot of the total number of genes per cell
sns.violinplot(data=adata.obs, y='n_genes')
plt.title('Total number of genes per cell')
plt.show()

In [ ]:
if 'GAPDH' in adata.var['gene_name'].values:
    print("GAPDH is in the dataset.")
else:
    print("GAPDH is not found in the dataset.")

In [ ]:
# Define the housekeeping genes
housekeeping_genes = ['ACTB', 'GAPDH', 'RPL13A', 'RPLP0', 'B2M', 'HPRT1',
                      'SDHA', 'TBP', 'GUSB', 'PPIA', 'TUBB', 'YWHAZ',
                      'PGK1', 'EIF4A2', 'RPS18', 'RPL32', 'RPS11', 'RPS13',
                      'RPL4', 'PPIB']

# Subset the list to the genes actually in the dataset
housekeeping_genes = [gene for gene in housekeeping_genes if gene in adata.var_names]

# Create a DataFrame for gene expressions
gene_expressions = pd.DataFrame({gene: adata[:, gene].X.toarray().flatten() for gene in housekeeping_genes})

# Melt the DataFrame to a long format for plotting
gene_expressions_melted = gene_expressions.melt(var_name='gene', value_name='expression')

# Create violin plots
plt.figure(figsize=(15, 10))
sns.violinplot(data=gene_expressions_melted, x='gene', y='expression', palette='Pastel1')
plt.xticks(rotation=90, fontsize=18, fontname='DejaVu Sans')
plt.yticks(fontsize=18, fontname='DejaVu Sans')
plt.xlabel('Gene', fontsize=18, fontname='DejaVu Sans')
plt.ylabel('Expression', fontsize=18, fontname='DejaVu Sans')
plt.title('Housekeeping Gene Expressions', fontsize=18, fontname='DejaVu Sans')
plt.show()

In [ ]:
# Define the housekeeping genes
housekeeping_genes = ['YWHAZ', 'PGK1', 'EIF4A2', 'RPS18', 'RPL32', 'RPS11', 'RPS13', 'RPL4', 'PPIB', 'HMBS']

# Subset the list to the genes actually in the dataset
housekeeping_genes = [gene for gene in housekeeping_genes if gene in adata.var_names]

# Create a DataFrame for gene expressions
gene_expressions = pd.DataFrame({gene: adata[:, gene].X.toarray().flatten() for gene in housekeeping_genes})

# Melt the DataFrame to a long format for plotting
gene_expressions_melted = gene_expressions.melt(var_name='gene', value_name='expression')

# Create violin plots
plt.figure(figsize=(15, 10))
sns.violinplot(data=gene_expressions_melted, x='gene', y='expression', palette='Pastel1')
plt.xticks(rotation=90, fontsize=18, fontname='DejaVu Sans')
plt.yticks(fontsize=18, fontname='DejaVu Sans')
plt.xlabel('Gene', fontsize=18, fontname='DejaVu Sans')
plt.ylabel('Expression', fontsize=18, fontname='DejaVu Sans')
plt.title('Housekeeping Gene Expressions', fontsize=18, fontname='DejaVu Sans')
plt.show()

In [ ]:
# Define the housekeeping genes
housekeeping_genes = ['RPS20', 'RPL27A', 'UBC', 'POLR2A', 'HSP90AB1', 'NONO', 'HNRNPH1', 'RPL7', 'RPL10A', 'RPL5']

# Subset the list to the genes actually in the dataset
housekeeping_genes = [gene for gene in housekeeping_genes if gene in adata.var_names]

# Create a DataFrame for gene expressions
gene_expressions = pd.DataFrame({gene: adata[:, gene].X.toarray().flatten() for gene in housekeeping_genes})

# Melt the DataFrame to a long format for plotting
gene_expressions_melted = gene_expressions.melt(var_name='gene', value_name='expression')

# Create violin plots
plt.figure(figsize=(15, 10))
sns.violinplot(data=gene_expressions_melted, x='gene', y='expression', palette='Pastel1')
plt.xticks(rotation=90, fontsize=18, fontname='DejaVu Sans')
plt.yticks(fontsize=18, fontname='DejaVu Sans')
plt.xlabel('Gene', fontsize=18, fontname='DejaVu Sans')
plt.ylabel('Expression', fontsize=18, fontname='DejaVu Sans')
plt.title('Housekeeping Gene Expressions', fontsize=18, fontname='DejaVu Sans')
plt.show()

In [ ]:
def plot_genes_violin(genes_list):
    # Create a DataFrame for gene expressions
    gene_expressions = pd.DataFrame({gene: adata[:, gene].X.toarray().flatten() for gene in genes_list})

    # Melt the DataFrame to a long format for plotting
    gene_expressions_melted = gene_expressions.melt(var_name='gene', value_name='expression')

    # Create violin plots
    plt.figure(figsize=(15, 10))
    sns.violinplot(data=gene_expressions_melted, x='gene', y='expression', palette='Pastel1')
    plt.xticks(rotation=90, fontsize=18, fontname='DejaVu Sans')
    plt.yticks(fontsize=18, fontname='DejaVu Sans')
    plt.xlabel('Gene', fontsize=18, fontname='DejaVu Sans')
    plt.ylabel('Expression', fontsize=18, fontname='DejaVu Sans')
    plt.title('Violin Plot of Housekeeping Gene Expressions', fontsize=18, fontname='DejaVu Sans')
    plt.show()

# Now let's start with the most important genes:
important_genes = ['GAPDH', 'ACTB', 'HPRT1', 'RPL13A', 'B2M',
                   'SDHA', 'RPLP0', 'GUSB', 'PPIA', 'TUBB']

# Ensure the genes are in the dataset
important_genes = [gene for gene in important_genes if gene in adata.var_names]

# Plot
plot_genes_violin(important_genes)

In [ ]:
import matplotlib.pyplot as plt

# Set global font size
plt.rcParams.update({'font.size': 14})

# Increase the figure size
fig, ax = plt.subplots(figsize=(10, 10))

# Increase DPI for higher resolution
fig.dpi = 200

# Create a UMAP for major_type with adjusted parameters
sc.pl.umap(adata, color='major_type', legend_loc='on data', title='Major Type', frameon=False, ax=ax, show=False)

# Adjust the title size
ax.set_title('Major Type', fontsize=20)

# Show the plot
plt.show()

In [ ]:
import scanpy as sc

# Perform t-SNE on the data
sc.tl.tsne(adata)

# Now you can plot
fig, ax = plt.subplots(figsize=(10, 10))
fig.dpi = 200
sc.pl.tsne(adata, color='major_type', legend_loc='on data', title='Major Type', frameon=False, ax=ax, show=False)
ax.set_title('Major Type', fontsize=20)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc

# Define the path where you want to save the HDF5 file
path_h5ad = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/obs_leidenclusters.h5ad"

# Load the AnnData object from an HDF5 file
adata = sc.read(path_h5ad)

# List of genes of interest
genes_of_interest = ['GP1BA', 'ITGA2B', 'PF4', 'VWF', 'MPL', 'CD4']

# Check if the genes are present in the data
missing_genes = [gene for gene in genes_of_interest if gene not in adata.var['gene_name'].values]
if missing_genes:
    print(f"These genes are not found in the dataset: {missing_genes}")

# Get the index of genes of interest in the dataframe
indices = [i for i, gene in enumerate(adata.var['gene_name']) if gene in genes_of_interest]

# Subset the data for the genes of interest
adata_subset = adata[:, indices]

# Convert the anndata to dataframe
df = adata_subset.to_df()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df, cmap='viridis')
plt.show()

## Save Merged Leiden CSV File as an H5AD file

In AnnData the main matrix is expected to be number, but here we have both numeric and non-numeric data. Therefore we will use .X for number data and .obs for string data in the dataframe.

In [ ]:
# Read the CSV File
df = pd.read_csv("/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/mergedleiden.csv")

# Select numeric columns for the .X part of the AnnData
numeric_columns = ['n_counts', 'n_genes', 'percent_mito']  # add more numeric column names if needed
X = df[numeric_columns].values

# Select non-numeric columns for the .obs part of the AnnData
obs = df.drop(numeric_columns, axis=1)

# Convert to AnnData
mergedlei = sc.AnnData(X=X, obs=obs)

# Write the AnnData to an HDF5 file
mergedlei.write("/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/mergedleiden.h5ad")


In [ ]:
# Display the head of the observations
mergedlei.obs.head()

Need to connect this to patient h5ad file

In [ ]:
# Define the path where you want to save the file
t2g_final = "/content/drive/MyDrive/Pate_Lab/DNP/Bioinformatics/H17_LungMk/Data_files/All Sample ID Data/t2g_final.h5ad"

# Read the data
adata3 = anndata.read(t2g_final)

In [ ]:
adata3.var

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc

# List of genes of interest
genes_of_interest = ['GP1BA', 'GP9', 'ITGA2B', 'ITGB3', 'PF4', 'VWF', 'SELP', 'CD9', 'FCER1G', 'ITGA2', 'ITGB1',
                     'F13A1', 'CD36', 'THBS1', 'FLI1', 'MPL', 'NF-E2', 'PF4V1', 'ITGAV', 'PECAM1', 'ITGAM', 'CD63',
                     'P2RY12', 'CD42B', 'CD42A', 'CD41', 'CD61', 'CD51', 'CD41B', 'CD34']

# Check if the genes are present in the data
missing_genes = [gene for gene in genes_of_interest if gene not in adata3.var['gene_name'].values]
if missing_genes:
    print(f"These genes are not found in the dataset: {missing_genes}")

# Get the index of genes of interest in the dataframe
indices = [i for i, gene in enumerate(adata3.var['gene_name']) if gene in genes_of_interest]

# Subset the data for the genes of interest
adata_subset = adata3[:, indices]

# Convert the anndata to dataframe
df = adata_subset.to_df()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df, cmap='viridis')
plt.show()
